In [25]:
# Install Libraries if unavailable
# !pip install keras
# !pip install tensorflow
# !pip install numpy
# !pip install pandas
# !pip install ekphrasis
# !pip install ftfy
# !pip install ujson
# !pip install textblob
# !pip install --upgrade pip

# Import Libraries

In [41]:
# from keras.models import Sequential
# from keras.layers import Dense
import numpy as np


# Import the dataset and unzip it

In [2]:
# !mkdir data &&wget http://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip && unzip smsspamcollection.zip -d data/

mkdir: cannot create directory ‘data’: File exists


In [42]:
from os import walk

f = []
d=[]
for (dirpath, dirnames, filenames) in walk("./data"):
    f.extend(filenames)
    d.extend(dirnames)
    break
print(f)
print(d)

['readme', 'SMSSpamCollection']
['dog-breed']


# Load the Dataset

In [43]:
import csv
raw_data = open("./data/SMSSpamCollection", 'rt')
dataset =list(csv.reader(raw_data,delimiter="\t"))

In [44]:
import pandas as pd
data=pd.DataFrame(dataset)

In [45]:
y=data.ix[:,0]

/home/controller/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


In [46]:
print(y)

0        ham
1        ham
2       spam
3        ham
4        ham
5       spam
6        ham
7        ham
8       spam
9       spam
10       ham
11      spam
12      spam
13       ham
14       ham
15      spam
16       ham
17       ham
18       ham
19      spam
20       ham
21       ham
22       ham
23       ham
24       ham
25       ham
26       ham
27       ham
28       ham
29       ham
        ... 
5542     ham
5543     ham
5544     ham
5545     ham
5546     ham
5547    spam
5548     ham
5549     ham
5550     ham
5551     ham
5552     ham
5553     ham
5554     ham
5555     ham
5556     ham
5557     ham
5558     ham
5559     ham
5560     ham
5561     ham
5562     ham
5563     ham
5564     ham
5565     ham
5566    spam
5567    spam
5568     ham
5569     ham
5570     ham
5571     ham
Name: 0, Length: 5572, dtype: object


In [47]:
X_words = data.ix[:,1]

from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_words, y, test_size = 0.2, random_state = 0)
# type(X_test)
# type(X_words)

In [20]:
# from keras.preprocessing.text import Tokenizer

# from ekphrasis.classes.preprocessor import TextPreProcessor
# from ekphrasis.classes.tokenizer import SocialTokenizer
# from ekphrasis.dicts.emoticons import emoticons

# text_processor = TextPreProcessor(
#     # terms that will be normalized
#     normalize=['url', 'email', 'percent', 'money', 'phone', 'user',
#         'time', 'url', 'date', 'number'],
#     # terms that will be annotated
#     annotate={},
#     fix_html=False,  # fix HTML tokens
    
#     # corpus from which the word statistics are going to be used 
#     # for word segmentation 
# #     segmenter="twitter", 
    
#     # corpus from which the word statistics are going to be used 
#     # for spell correction
#     corrector="twitter", 
    
#     unpack_hashtags=False,  # perform word segmentation on hashtags
#     unpack_contractions=False,  # Unpack contractions (can't -> can not)
#     spell_correct_elong=True,  # spell correction for elongated words
    
#     # select a tokenizer. You can use SocialTokenizer, or pass your own
#     # the tokenizer, should take as input a string and return a list of tokens
# #     tokenizer=SocialTokenizer(lowercase=True).tokenize,
    
#     # list of dictionaries, for replacing tokens extracted from the text,
#     # with other expressions. You can pass more than one dictionaries.
# #     dicts=[emoticons]
# )

Reading twitter - 1grams ...


In [48]:
# from keras.preprocessing.sequence import pad_sequences
import re
# import nltk
# nltk.download()
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import *
stop_words=stopwords.words("english")
porter = PorterStemmer()

# Create a Pre-Processing Step - Replace Phone Numbers, Email IDs and Web Pages to a standard Text

In [49]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
from textblob import TextBlob
# cv = CountVectorizer(analyzer=split_into_lemmas).fit(X_train)


def preprocess_text(message):
    message = message.lower()
    cleaned = re.sub(r'\b[\w\-.]+?@\w+?\.\w{2,4}\b', ' emailaddr ', message)
    cleaned = re.sub(r'(http[s]?\S+)|(\w+\.[A-Za-z]{2,4}\S*)', ' httpaddr ',
                     cleaned)
    cleaned = re.sub(r'£|\$', 'moneysymb', cleaned)
    cleaned = re.sub(
        r'\b(\+\d{1,2}\s)?\d?[\-(.]?\d{3}\)?[\s.-]?\d{3}[\s.-]?\d{4}\b',
        ' phonenumbr ', cleaned)
    cleaned = re.sub(r'\d+(\.\d+)?', ' numbr ', cleaned)
    cleaned = re.sub(r'[^\w\d\s]', ' ', cleaned)
    cleaned = re.sub(r'\s+', ' ', cleaned)
    cleaned = re.sub(r'^\s+|\s+?$', '', cleaned)
    cleaned = cleaned.split()
    ps = PorterStemmer()
    cleaned = [ps.stem(word) for word in cleaned if not word in set(stop_words)]
    cleaned = ' '.join(cleaned)
    return cleaned

In [50]:
# X_train_preprocessed = []
# for sentence in X_train.values.tolist():
#     X_train_preprocessed.append(preprocess_text(sentence))

# dummy_set = ("The sky is blue.", "The sun is bright.")

# ?CountVectorizer()
cv = CountVectorizer(preprocessor=preprocess_text)
cv.fit_transform(X_train)
print(cv.vocabulary_)
# print(X_train_preprocessed[0])
# print(type(X_train_preprocessed))
# print(type(X_train_preprocessed[0]))

{'good': 1928, 'movi': 3046, 'ok': 3303, 'leav': 2636, 'hourish': 2192, 'free': 1762, 'give': 1896, 'otherwis': 3372, 'nalla': 3106, 'adi': 57, 'entey': 1451, 'nattil': 3122, 'kittum': 2555, 'emigr': 1425, 'someth': 4393, 'mayb': 2886, 'numbr': 3255, 'bit': 495, 'hope': 2176, 'got': 1944, 'home': 2159, 'babe': 360, 'still': 4532, 'awak': 344, 'kay': 2516, 'sinc': 4283, 'alreadi': 147, 'workin': 5390, 'get': 1875, 'job': 2452, 'said': 4049, 'matter': 2878, 'mind': 2948, 'say': 4088, 'oh': 3298, 'yeah': 5478, 'diet': 1215, 'flew': 1688, 'window': 5336, 'sorri': 4415, 'thing': 4809, 'may': 2885, 'pub': 3748, 'later': 2613, 'ill': 2273, 'call': 678, 'even': 1498, 'idea': 2253, 'dear': 1123, 'reach': 3841, 'room': 4001, 'look': 2733, 'scrumptiou': 4113, 'daddi': 1088, 'want': 5220, 'eat': 1375, 'night': 3186, 'long': 2730, 'badrith': 368, 'chennai': 793, 'sure': 4634, 'pick': 3534, 'us': 5097, 'competit': 924, 'da': 1083, 'car': 710, 'park': 3440, 'okay': 3304, 'soon': 4406, 'best': 461, 'y

In [51]:
# X_words.iloc[0]
X_train_preprocessed = cv.transform(X_train)
X_train_preprocessed.todense()
print(X_train_preprocessed)

  (0, 1928)	1
  (0, 2192)	1
  (0, 2636)	1
  (0, 3046)	1
  (0, 3303)	1
  (1, 57)	1
  (1, 1451)	1
  (1, 1762)	1
  (1, 1896)	1
  (1, 2555)	1
  (1, 3106)	1
  (1, 3122)	1
  (1, 3372)	1
  (2, 495)	1
  (2, 1425)	1
  (2, 2176)	1
  (2, 2886)	1
  (2, 3255)	1
  (2, 3303)	1
  (2, 4393)	1
  (3, 344)	1
  (3, 360)	1
  (3, 1944)	1
  (3, 2159)	1
  (3, 4532)	1
  :	:
  (4454, 727)	1
  (4454, 762)	1
  (4454, 1069)	1
  (4454, 1502)	1
  (4454, 2208)	1
  (4454, 3014)	1
  (4454, 3255)	4
  (4454, 3526)	1
  (4454, 5003)	1
  (4454, 5086)	1
  (4454, 5331)	1
  (4454, 5358)	2
  (4455, 1361)	1
  (4455, 1912)	1
  (4455, 2193)	1
  (4455, 2900)	1
  (4455, 3255)	1
  (4455, 4060)	1
  (4456, 1123)	1
  (4456, 1617)	1
  (4456, 1875)	1
  (4456, 2984)	1
  (4456, 4189)	1
  (4456, 4322)	1
  (4456, 4882)	1


In [52]:
#Create Bag og Words
# del X
# from textblob import TextBlob
# def split_into_lemmas(message):
#     message = message.lower()
#     words = TextBlob(message).words
#     # for each word, take its "base form" = lemma 
#     return [word.lemma for word in words]
tfidf = TfidfTransformer(norm="l2")
tfidf.fit(X_train_preprocessed)
print("IDF:", tfidf.idf_)

IDF: [ 8.30384323  8.01616115  8.70930833 ...,  8.70930833  8.70930833
  8.70930833]


In [53]:
X_train_tfidf = tfidf.transform(X_train_preprocessed)
print(X_train_tfidf.todense())

[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]


In [78]:
# from sklearn.naive_bayes import GaussianNB
# classifier = GaussianNB()
# from sklearn.ensemble import RandomForestClassifier
# classifier = RandomForestClassifier(n_estimators = 2, criterion = 'entropy', random_state = 0)
# from sklearn.svm import SVC
# classifier = SVC(kernel = 'rbf', random_state = 0)
# from sklearn.neighbors import KNeighborsClassifier
# classifier = KNeighborsClassifier(n_neighbors = 2, metric = 'minkowski', p = 2)
classifier.fit(X_train_tfidf, y_train)

GaussianNB(priors=None)

In [79]:
X_test_preprocessed = cv.transform(X_test)
X_test_preprocessed.todense()
X_test_tfidf = tfidf.transform(X_test_preprocessed)

In [81]:
y_test_pred = classifier.predict(X_test_tfidf)

In [82]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_test_pred)

In [83]:
print(cm)

[[820 135]
 [ 18 142]]


In [74]:
y_train_pred=classifier.predict(X_train_tfidf)

In [75]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_train, y_train_pred)

In [54]:
X_preprocessed_test = []
for sentence in X_test.values.tolist():
    X_preprocessed_test.append(preprocess_text(sentence))

In [55]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 1500)
X_test_preprocessed = cv.fit_transform(X_preprocessed_test)

In [56]:
y_test_pred=classifier.predict(X_test_preprocessed)

In [57]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_test_pred)

In [58]:
print(cm)

[[955   0]
 [160   0]]


In [61]:
X_test[y_test!=y_test_pred]

690     <Forwarded from 448712404000>Please CALL 08712...
3587    I am hot n horny and willing I live local to y...
134     Sunshine Quiz Wkly Q! Win a top Sony DVD playe...
15      XXXMobileMovieClub: To use your credit, click ...
3155    Not heard from U4 a while. Call 4 rude chat pr...
4579    Urgent! call 09066350750 from your landline. Y...
2311    You have won a Nokia 7250i. This is what you g...
3424    Had your mobile 10 mths? Update to latest Oran...
42      07732584351 - Rodger Burns - MSG = We tried to...
3556    From next month get upto 50% More Calls 4 Ur s...
684     Hi I'm sue. I am 20 years old and work as a la...
3595    Do you want a New Nokia 3510i Colour Phone Del...
1536    You have won a Nokia 7250i. This is what you g...
1196    You have 1 new voicemail. Please call 08719181503
1741    UR GOING 2 BAHAMAS! CallFREEFONE 08081560665 a...
3978    Great NEW Offer - DOUBLE Mins & DOUBLE Txt on ...
3433    Camera - You are awarded a SiPix Digital Camer...
4577    Congra

In [66]:
y_train.value_counts()

ham     3870
spam     587
Name: 0, dtype: int64